In [26]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


In [27]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [28]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [29]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')


soup ready


In [30]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [31]:
table_rows = table.find_all('tr')

#table_rows

In [60]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(5))

  PostalCode           Borough     Neighbourhood
1        M1A      Not assigned      Not assigned
2        M2A      Not assigned      Not assigned
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront


## Data Transformed into DataFrame

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [34]:
df.shape

(287, 3)

In [61]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [73]:
df1 = df.reset_index()
df1.head()
df1.columns
df1.drop('index', axis=1, inplace=True)

Index(['index', 'PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [81]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## Question 2 
(using the Geospatial_Coordiante.csv)


In [83]:
!pip install geopy

In [85]:
data2 = pd.read_csv("https://cocl.us/Geospatial_data") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [86]:
data2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#data2.head()

In [275]:
data = pd.merge(df1, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


## Question 3
analyzing Borough

In [104]:
CLIENT_ID = 'VO3CR5T34VIYBYGSOK4QWJ30HTRNX4N3YLRODMXLYK4CRWTH' # your Foursquare ID
CLIENT_SECRET = 'INL4034YUC0HEUYMEV40OAHSSOIMFOHZO5XWDNS5I3YVSAHX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VO3CR5T34VIYBYGSOK4QWJ30HTRNX4N3YLRODMXLYK4CRWTH
CLIENT_SECRET:INL4034YUC0HEUYMEV40OAHSSOIMFOHZO5XWDNS5I3YVSAHX


In [89]:
data.loc[0, 'Borough']

'Scarborough'

In [91]:
Borough_latitude = data.loc[0, 'Latitude'] # Borough latitude value
Borough_longitude = data.loc[0, 'Longitude'] # Borough longitude value

Borough_name = data.loc[0, 'Borough'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(Borough_name, 
                                                               Borough_latitude, 
                                                               Borough_longitude))



Latitude and longitude values of Scarborough are 43.806686299999996, -79.19435340000001.


In [115]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Borough_latitude, 
    Borough_longitude, 
    radius, 
    LIMIT)
url # display UR

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e512fad1a4b0a001b3e523e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.806704300018,
    'lng': -79.19432850472293},
   'sw': {'lat': 43.80666829998199, 'lng': -79.19437829527709}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [103]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

KeyError: 'groups'

In [110]:
# function that extracts the category of the town
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [152]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [153]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

NameError: name 'nearby_venues' is not defined

In [154]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

    # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [156]:
Toronto_venues = getNearbyVenues(names= data['Borough'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )



Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central T

### let's check the size of the resulting dataframe

In [157]:
print(Toronto_venues.shape)
Toronto_venues.head()

(262, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
1,Scarborough,43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant
2,Scarborough,43.799525,-79.318389,Price Chopper,43.799445,-79.318563,Grocery Store
3,Scarborough,43.799525,-79.318389,Subway,43.798671,-79.318475,Sandwich Place
4,Scarborough,43.799525,-79.318389,Shoppers Drug Mart,43.799966,-79.317985,Pharmacy


In [158]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,24,24,24,24,24,24
Downtown Toronto,164,164,164,164,164,164
East Toronto,1,1,1,1,1,1
East York,5,5,5,5,5,5
Etobicoke,4,4,4,4,4,4
Mississauga,1,1,1,1,1,1
North York,33,33,33,33,33,33
Scarborough,10,10,10,10,10,10
West Toronto,20,20,20,20,20,20


### Let's find out how many unique categories can be curated from all the returned venues

In [159]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 78 uniques categories.


## Analyze Each Neighborhood

In [160]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
Toronto_onehot.shape

(262, 79)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [162]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Central Toronto,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.041667,...,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,0.000000,0.0
1,Downtown Toronto,0.012195,0.02439,0.0,0.02439,0.000000,0.02439,0.000000,0.0,0.012195,...,0.030488,0.018293,0.018293,0.012195,0.000000,0.018293,0.000000,0.0,0.018293,0.0
2,East Toronto,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0
3,East York,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,Etobicoke,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
5,Mississauga,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
6,North York,0.000000,0.00000,0.0,0.00000,0.090909,0.00000,0.030303,0.0,0.000000,...,0.060606,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.000000,0.0
7,Scarborough,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
8,West Toronto,0.000000,0.00000,0.1,0.00000,0.000000,0.10000,0.000000,0.1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.1


##  Let's confirm the new size

In [163]:
Toronto_grouped.shape

(9, 79)

## Let's print each neighborhood along with the top 5 most common venues


In [165]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
                    venue  freq
0            Liquor Store  0.21
1             Supermarket  0.21
2            Dessert Shop  0.08
3                    Park  0.08
4  Furniture / Home Store  0.08


----Downtown Toronto----
                   venue  freq
0            Coffee Shop  0.13
1                   Café  0.05
2  Performing Arts Venue  0.05
3                    Gym  0.04
4          Deli / Bodega  0.04


----East Toronto----
                 venue  freq
0                Trail   1.0
1  American Restaurant   0.0
2   Mac & Cheese Joint   0.0
3                 Pool   0.0
4           Playground   0.0


----East York----
                 venue  freq
0          Coffee Shop   0.2
1  Housing Development   0.2
2    Indian Restaurant   0.2
3       Sandwich Place   0.2
4  Sporting Goods Shop   0.2


----Etobicoke----
                  venue  freq
0          Dance Studio  0.50
1           Coffee Shop  0.25
2           Pizza Place  0.25
3  Gym / Fitness Center  0.00
4         

### Let's put that into a *pandas* dataframe

In [166]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [290]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Supermarket,Liquor Store,Furniture / Home Store,Dessert Shop,Park,Italian Restaurant,Coffee Shop,Seafood Restaurant,Bowling Alley,Bookstore
1,Downtown Toronto,Coffee Shop,Café,Performing Arts Venue,Gym,Deli / Bodega,Sushi Restaurant,Art Gallery,Concert Hall,Bakery,Cocktail Bar
2,East Toronto,Trail,Yoga Studio,Diner,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
3,East York,Coffee Shop,Sporting Goods Shop,Sandwich Place,Housing Development,Indian Restaurant,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega
4,Etobicoke,Dance Studio,Coffee Shop,Pizza Place,Diner,College Gym,Concert Hall,Cosmetics Shop,Deli / Bodega,Dessert Shop,Yoga Studio


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [284]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
Toronto_grouped_clustering

,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Bowling Alley,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.041667,0.041667,...,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,0.000000,0.0
1,0.012195,0.02439,0.0,0.02439,0.000000,0.02439,0.000000,0.0,0.012195,0.000000,...,0.030488,0.018293,0.018293,0.012195,0.000000,0.018293,0.000000,0.0,0.018293,0.0
2,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0
3,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
5,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
6,0.000000,0.00000,0.0,0.00000,0.090909,0.00000,0.030303,0.0,0.000000,0.000000,...,0.060606,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.000000,0.0
7,0.000000,0.00000,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
8,0.000000,0.00000,0.1,0.00000,0.000000,0.10000,0.000000,0.1,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.1


### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [295]:


Toronto_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted)

Toronto_merged.set_index("Neighbourhood") # check the last columns!

,PostalCode,Borough,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighbourhood,,,,,,,,,,,,,,,,
Rouge,M1B,Scarborough,43.806686,-79.194353,1.0,Central Toronto,Supermarket,Liquor Store,Furniture / Home Store,Dessert Shop,Park,Italian Restaurant,Coffee Shop,Seafood Restaurant,Bowling Alley,Bookstore
Malvern,M1B,Scarborough,43.806686,-79.194353,1.0,Downtown Toronto,Coffee Shop,Café,Performing Arts Venue,Gym,Deli / Bodega,Sushi Restaurant,Art Gallery,Concert Hall,Bakery,Cocktail Bar
Highland Creek,M1C,Scarborough,43.784535,-79.160497,2.0,East Toronto,Trail,Yoga Studio,Diner,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
Rouge Hill,M1C,Scarborough,43.784535,-79.160497,4.0,East York,Coffee Shop,Sporting Goods Shop,Sandwich Place,Housing Development,Indian Restaurant,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega
Port Union,M1C,Scarborough,43.784535,-79.160497,3.0,Etobicoke,Dance Studio,Coffee Shop,Pizza Place,Diner,College Gym,Concert Hall,Cosmetics Shop,Deli / Bodega,Dessert Shop,Yoga Studio
Guildwood,M1E,Scarborough,43.763573,-79.188711,0.0,Mississauga,Coffee Shop,Electronics Store,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Yoga Studio
Morningside,M1E,Scarborough,43.763573,-79.188711,1.0,North York,Coffee Shop,Fried Chicken Joint,Theater,Bank,Fast Food Restaurant,Deli / Bodega,Italian Restaurant,Clothing Store,Hot Dog Joint,Restaurant
West Hill,M1E,Scarborough,43.763573,-79.188711,1.0,Scarborough,Grocery Store,Electronics Store,Chinese Restaurant,Pharmacy,Sandwich Place,Cosmetics Shop,Bubble Tea Shop,Playground,Coffee Shop,Fast Food Restaurant
Woburn,M1G,Scarborough,43.770992,-79.216917,1.0,West Toronto,Yoga Studio,Beer Store,Coffee Shop,Record Shop,Mac & Cheese Joint,Japanese Restaurant,Brewery,Pizza Place,Asian Restaurant,Bar


In [305]:
Toronto_merged = Toronto_merged.set_index('Neighbourhood'), on='Neighbourhood'

SyntaxError: can't assign to function call (<ipython-input-305-2c3f15e5044f>, line 1)

In [307]:
Toronto_merged['Cluster_Labels'] = Toronto_merged.Cluster_Labels.astype(float)

AttributeError: 'DataFrame' object has no attribute 'Cluster_Labels'

### Finally, let's visualize the resulting clusters

In [296]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: cannot convert the series to <class 'float'>

#  Examine Clusters


In [308]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0.0,Mississauga,Coffee Shop,Electronics Store,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Yoga Studio


## Cluster 2

In [310]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Central Toronto,Supermarket,Liquor Store,Furniture / Home Store,Dessert Shop,Park,Italian Restaurant,Coffee Shop,Seafood Restaurant,Bowling Alley,Bookstore
1,Scarborough,1.0,Downtown Toronto,Coffee Shop,Café,Performing Arts Venue,Gym,Deli / Bodega,Sushi Restaurant,Art Gallery,Concert Hall,Bakery,Cocktail Bar
6,Scarborough,1.0,North York,Coffee Shop,Fried Chicken Joint,Theater,Bank,Fast Food Restaurant,Deli / Bodega,Italian Restaurant,Clothing Store,Hot Dog Joint,Restaurant
7,Scarborough,1.0,Scarborough,Grocery Store,Electronics Store,Chinese Restaurant,Pharmacy,Sandwich Place,Cosmetics Shop,Bubble Tea Shop,Playground,Coffee Shop,Fast Food Restaurant
8,Scarborough,1.0,West Toronto,Yoga Studio,Beer Store,Coffee Shop,Record Shop,Mac & Cheese Joint,Japanese Restaurant,Brewery,Pizza Place,Asian Restaurant,Bar


## Cluster 3

In [311]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2.0,East Toronto,Trail,Yoga Studio,Diner,Coffee Shop,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop


## Cluster 4

In [312]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3.0,Etobicoke,Dance Studio,Coffee Shop,Pizza Place,Diner,College Gym,Concert Hall,Cosmetics Shop,Deli / Bodega,Dessert Shop,Yoga Studio


## Cluster 5

In [318]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4.0,East York,Coffee Shop,Sporting Goods Shop,Sandwich Place,Housing Development,Indian Restaurant,College Gym,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega
